In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


modelPath = "T:/CodeBase/Python/Computer Vision/MID/Project/FaceDetection/trainningCodeM.h5"
classifier = load_model(modelPath)


class_labels = ["Touhid", "Tishad", "Abir", "Shimla"]


imgSize = 64


haar_cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
object_cascade = cv2.CascadeClassifier(haar_cascade_path)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  
    objects = object_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in objects:
        # Extract the Region of Interest (ROI)
        roi = gray[y:y+h, x:x+w]

        #Resize the ROI to match the model's input size
        roi_resized = cv2.resize(roi, (imgSize, imgSize))

        #Normalize the pixel values
        roi_normalized = roi_resized / 255.0

        #Reshape the ROI for the model (batch_size=1, height=imgSize, width=imgSize, channels=1)
        roi_reshaped = roi_normalized.reshape(1, imgSize, imgSize, 1)

        #prediction
        predictions = classifier.predict(roi_reshaped)
        predicted_class = np.argmax(predictions)
        confidence = predictions[0][predicted_class] * 100

        
        if confidence > 50:
            label = class_labels[predicted_class]
            label_text = f"{label}: {confidence:.2f}%"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

 
    cv2.imshow("CVPR Groupmate Detection", frame)

  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
